<a href="https://colab.research.google.com/github/Darshan0902/EDUMITRA-./blob/main/IF_TDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv("/content/MOCK_DATA (3).csv")

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming df is your DataFrame
# df = pd.DataFrame(...)

label_encoders = {}
for col in ['CITY ', 'SKILLS', 'USER NAME']:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

df['SKILLS'] = label_encoders['SKILLS'].inverse_transform(df['SKILLS'])

user_names_mapping = dict(zip(label_encoders['USER NAME'].transform(df['USER NAME']), df['USER NAME']))

X = df.drop(columns=['USER NAME'])
y = df['USER NAME']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer()
skills_matrix = tfidf_vectorizer.fit_transform(df['SKILLS'])

cosine_sim = cosine_similarity(skills_matrix, skills_matrix)

def recommend_users(selected_skills):
    selected_skills_vector = tfidf_vectorizer.transform([selected_skills])
    cosine_sim_scores = cosine_similarity(selected_skills_vector, skills_matrix).flatten()
    user_indices = cosine_sim_scores.argsort()[:-6:-1]  # Get top 5 most similar users
    recommended_user_indices = df.iloc[user_indices]['USER NAME'].values
    recommended_users = [user_names_mapping[user] for user in recommended_user_indices]
    return recommended_users

selected_skills = 'PHP'
recommended_users = recommend_users(selected_skills)
print("Recommended users for skills '{}':".format(selected_skills))
print(recommended_users)


Recommended users for skills 'PHP':
[4, 2, 19, 22, 18]


# IDs showing.

In [ ]:
def recommend_users(selected_skills):
    selected_skills_vector = tfidf_vectorizer.transform([selected_skills])
    cosine_sim_scores = cosine_similarity(selected_skills_vector, skills_matrix).flatten()
    user_indices = cosine_sim_scores.argsort()[:-6:-1]  # Get top 5 most similar users
    recommended_user_indices = df.iloc[user_indices]['USER NAME'].values
    recommended_user_names = [user_names_mapping[user] for user in recommended_user_indices]
    return list(zip(user_indices, recommended_user_names))


In [ ]:
selected_skills = 'PHP'
recommended_users = recommend_users(selected_skills)
print("Recommended users for skills '{}':".format(selected_skills))
for user_id, user_name in recommended_users:
    print(f"{user_name} (ID: {user_id})")


Recommended users for skills 'PHP':
4 (ID: 427)
2 (ID: 323)
19 (ID: 605)
22 (ID: 582)
18 (ID: 715)


# Names testing :

In [ ]:
def recommend_users(selected_skills):
    selected_skills_vector = tfidf_vectorizer.transform([selected_skills])
    cosine_sim_scores = cosine_similarity(selected_skills_vector, skills_matrix).flatten()
    user_indices = cosine_sim_scores.argsort()[:-6:-1]  # Get top 5 most similar users
    recommended_user_names = df.iloc[user_indices]['USER NAME'].values
    return list(zip(recommended_user_names, cosine_sim_scores[user_indices]))

selected_skills = 'PHP'
recommended_users = recommend_users(selected_skills)
print("Recommended users for skills '{}':".format(selected_skills))
for user_name, similarity_score in recommended_users:
    print(f"User Name: {user_name}, Similarity Score: {similarity_score}")


Recommended users for skills 'PHP':
User Name: 4, Similarity Score: 1.0
User Name: 2, Similarity Score: 1.0
User Name: 19, Similarity Score: 1.0
User Name: 22, Similarity Score: 1.0
User Name: 18, Similarity Score: 1.0


# New Code testing 🐾


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Assuming df is the DataFrame containing the data

# Encode categorical variables
label_encoders = {}
for col in ['CITY ', 'SKILLS', 'USER NAME']:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Convert integer labels back to skill names
df['SKILLS'] = label_encoders['SKILLS'].inverse_transform(df['SKILLS'])

# Store original user names along with encoded counterparts
user_names_mapping = dict(zip(df['USER NAME'], df['USER NAME']))

# Split data into train and test sets
X = df.drop(columns=['USER NAME'])
y = df['USER NAME']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TF-IDF vectorizer for skills
tfidf_vectorizer = TfidfVectorizer()
skills_matrix = tfidf_vectorizer.fit_transform(df['SKILLS'])

# Train a k-nearest neighbors model
k = 5  # Number of neighbors to consider
knn_model = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model.fit(skills_matrix)

# Function to recommend users based on selected skills using KNN
def recommend_users_knn(selected_skills):
    selected_skills_vector = tfidf_vectorizer.transform([selected_skills])
    _, user_indices = knn_model.kneighbors(selected_skills_vector)
    recommended_user_indices = df.iloc[user_indices[0]]['USER NAME'].values
    recommended_users = [user_names_mapping[user] for user in recommended_user_indices]
    return recommended_users

# Example usage:
selected_skills = 'PHP'
recommended_users = recommend_users_knn(selected_skills)
print("Recommended users for skills '{}':".format(selected_skills))
print(recommended_users)


Recommended users for skills 'PHP':
[19, 14, 14, 18, 3]


Recommended users for skills 'PHP':
[19, 14, 14, 18, 3]


# 23-02-2024 ❎

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv("/content/MOCK_DATA.csv")

In [8]:
from collections import defaultdict

# Create a dictionary to store users indexed by their skills
users_by_skill = defaultdict(list)
for _, row in df.iterrows():
    users_by_skill[row['Skills']].append(row['id'])

# Function to recommend users based on common skills
def recommend_users_for_skill(skill):
    # Find users who have the specified skill
    users_with_skill = users_by_skill.get(skill, [])
    return users_with_skill

# Get user input for skill
selected_skill = input("Select your skill: ")

# Get recommended users for the selected skill
recommended_user_ids = recommend_users_for_skill(selected_skill)
recommended_users_info = df[df['id'].isin(recommended_user_ids)][['id', 'first_name', 'email', 'gender']]

# Print recommended users
if not recommended_users_info.empty:
    print(f"Users with skill '{selected_skill}':")
    print(recommended_users_info)
else:
    print(f"No users found with skill '{selected_skill}'.")


Select your skill: R
Users with skill 'R':
      id  first_name                           email   gender
8      9      Amelie              arivenzon8@epa.gov   Female
73    74       Randi                rruddom21@wp.com     Male
92    93   Melisande        mtilliards2k@myspace.com   Female
112  113       Rodie    rtuckett34@independent.co.uk   Female
127  128        Niel             njacquemot3j@a8.net     Male
267  268       Kathy               knoen7f@topsy.com   Female
300  301     Marleah         mmacari8c@amazonaws.com   Female
342  343        Lori       ltimpany9i@craigslist.org   Female
360  361       Golda               gblasea0@ehow.com   Female
392  393   Nathanael               nrosseyaw@aol.com     Male
395  396     Emelyne       etyghtaz@wunderground.com   Female
446  447        Dory         dedysonce@aboutads.info     Male
537  538      Sigrid               smcroryex@epa.gov   Female
573  574     Rosalyn            rtraytefx@weebly.com   Female
574  575      Shelba       